In [ ]:
from glob import glob
import satpy
import xarray as xr
from pyresample.geometry import AreaDefinition, create_area_def, SwathDefinition
from datetime import datetime
from mast import utils

In [ ]:
scene=glob ('/home/bornagain/backup500/MYD*')
#scene.pop(4)

In [ ]:
scene = glob('/home/bornagain/mounting_point/osvaldo/storage/WORK/DATA/MODIS/MOD05_L2/2020/**/*.hdf',recursive=True)

In [ ]:
len(scene)

In [ ]:
area_id = 'thule'
description = 'Thule ice box'
proj_id = 'thule'
center = (-68,76)
radius = (1000,1000)
shape = (19,19)
resolution =1000

proj_dict = {'proj': 'tmerc', 'lat_0': center[1], 'lon_0': center[0] , 'lat_ts':center[1], 'a': 6371228.0, 'units': 'm'}

center = xr.DataArray([center[0], center[1]], attrs={"units": "degrees"})
area_def = AreaDefinition.from_area_of_interest(area_id, proj_dict, shape, center, resolution)

datasets=[]

In [ ]:
def run_cycle(element):
    try:
        scn = satpy.Scene([element], reader='modis_l2')
        scn.load(['water_vapor_infrared'])
        local_scn = scn.resample(area_def, nprocs=16, radius_of_influence=10000)
        local_scn_xr= local_scn.to_xarray_dataset()
        local_scn_xr=local_scn_xr.expand_dims(time=[local_scn_xr.start_time])
        datasets.append(local_scn_xr)
    except(ValueError):
            #print(Exception)
            pass
array=xr.concat(datasets,dim='time')

In [ ]:
%%time
for element in scene:
    run_cycle(element)

In [ ]:
%%time
utils.parallel.parallel_CPU(run_cycle,scene)

In [ ]:
len(datasets)

In [ ]:
array

In [ ]:
variance = array.water_vapor_infrared.var(skipna=True)

In [ ]:
variance.values